In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from liams_funcs import *
from skimage import io
from PIL.Image import fromarray


In [ ]:

def show_image(image_path, save_to=None):
    image = io.imread(image_path)
    plt.imshow(image)
    plt.axis('off')
    if save_to:
        plt.savefig(save_to, bbox_inches='tight')
    plt.show()

# Define directories
control_dir = '../control_images/'
penetramax_dir = '../penetramax_images/'

# List files in directories
control_fnames = [control_dir + fname for fname in os.listdir(control_dir)]
penetramax_fnames = [penetramax_dir + fname for fname in os.listdir(penetramax_dir)]

# Debug: print the filenames to ensure they are correct
print("Control filenames:", control_fnames)
print("Penetramax filenames:", penetramax_fnames)

# Show and save the first image from control_fnames
show_image(control_fnames[0], save_to='test.png')


In [ ]:
dir_hist = '../corrected_images/hist_correction/'
dir_adapthist = '../corrected_images/adapthist_correction/'

In [ ]:

for fname in control_fnames + penetramax_fnames:
    final_path = dir_hist + fname.split('/')[-1]

    im = perform_hist_equalisation(plt.imread(fname))
    im = (255 * im).astype(np.uint8)

    fromarray(im).save(final_path)

for fname in control_fnames + penetramax_fnames:
    final_path = dir_adapthist + fname.split('/')[-1]

    im = perform_adapthist_equalisation(plt.imread(fname))
    im = (255 * im).astype(np.uint8)

    fromarray(im).save(final_path)

In [ ]:
fname = control_fnames[0].split('/')[-1]

path1 = control_dir + fname
path2 = dir_hist + fname
path3 = dir_adapthist + fname

show_image(path1)
show_image(path2)
show_image(path3)

In [ ]:
fname = penetramax_fnames[0].split('/')[-1]

path1 = penetramax_dir + fname
path2 = dir_hist + fname
path3 = dir_adapthist + fname

show_image(path1)
show_image(path2)
show_image(path3)

In [ ]:
from cellpose import io, models

diameters = np.arange(10, 200+1, 10)
model = models.Cellpose(model_type='nuclei', gpu=True)

In [ ]:
N_segments = []
Areas = []
for diameter in tqdm(diameters):
    n_segments = []
    areas = []
    for fname in control_fnames[:1] + penetramax_fnames[:1]:
        image = io.imread(dir_hist + fname.split('/')[-1])

        masks = model.eval(image, channels=[1,0], diameter=diameter)[0]

        n_segments.append(masks.max())
        areas.append(np.sum(masks != 0))

    N_segments.append(n_segments)
    Areas.append(areas)

N_segments = np.asarray(N_segments)
Areas = np.asarray(Areas)

In [ ]:
fig, (l, r) = plt.subplots(1, 2, figsize=(12,4))

l.set_prop_cycle(plt.cycler('color', plt.cm.Reds(np.linspace(0.5, 1, len(control_fnames)))))
r.set_prop_cycle(plt.cycler('color', plt.cm.Blues(np.linspace(0.5, 1, len(control_fnames)))))


l.plot(diameters, N_segments, '-o')
l.plot(diameters, N_segments.mean(axis=1), c='aqua', linewidth=3)
l.set_yscale('log'); l.grid(axis='y')
l.set_xlim(diameters[0], diameters[-1])#; l.set_ylim(0)
l.set_yticks([10, 50, 100, 200, 500], [10, 50, 100, 200, 500])
l.set_xlabel('Diameter (pix)'); l.set_ylabel('# of segments')

r.plot(diameters, Areas, '-o')
r.plot(diameters, Areas.mean(axis=1), c='gold', linewidth=3)
r.set_yscale('log')
r.set_xlim(diameters[0], diameters[-1])#; r.set_ylim(0)
r.set_xlabel('Diameter (pix)'); r.set_ylabel(r'Area covered (pix$^2$)')

fig.tight_layout()
fig.savefig('diameter_analysis.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
def rng_colour():
    from numpy.random import choice
    from numpy import arange, uint8

    R = choice(arange(256, dtype=uint8))
    G = choice(arange(256, dtype=uint8))
    B = choice(arange(256, dtype=uint8))

    return R, G, B

def plot_image_with_masks(image, masks):
    from numpy import copy, arange, stack
    from matplotlib.pyplot import subplots, show

    xx, yy = np.meshgrid(arange(image.shape[0]), arange(image.shape[1]))

    fig, ax = subplots(figsize=(8,8), dpi=300)

    ax.imshow(image, origin='lower')
    ax.contour(xx, yy, masks, levels=arange(masks.max()+1))

    show()

In [ ]:
all_masks = []
for diam in [25, 50, 75, 100]:
    image = io.imread(dir_adapthist + control_fnames[0].split('/')[-1])
    masks = model.eval(image, channels=[1,0], diameter=diam)[0]

    plot_image_with_masks(image, masks)
    all_masks.append(masks)

In [ ]:
def histogram_roundnesses(masks, bins:int=20, density:bool=False):
    from numpy import arange, histogram

    rs = []
    for val in arange(1, masks.max()+0.1):
        mask = (masks == val)
        mask = remove_borders(mask)
        rs.append(evaluate_roundness(mask))

    vals, edges = histogram(rs, range=(0.4,1), bins=bins, density=density)

    plt.figure()
    plt.stairs(vals, edges)
    plt.show()

In [ ]:
histogram_roundnesses(all_masks[0])
histogram_roundnesses(all_masks[1])
histogram_roundnesses(all_masks[2])
histogram_roundnesses(all_masks[3])

In [ ]:
def binary_dilateNtimes(mask, N:int=1):
    from skimage.morphology import dilation

    if N == 0:
        return mask
    return binary_dilateNtimes(dilation(mask), N=int(N-1))

def sequential_segmentation(fname, diams=[80, 80], N_dilations=5, threshold=0, show=False, save_to=None):
    from numpy import copy, uint8, arange, meshgrid
    from skimage.exposure import equalize_adapthist
    from cellpose import io

    image = io.imread(fname)
    im = copy(image)
    im[:,:,0] = (255 * equalize_adapthist(im[:,:,0])).astype(uint8)
    xx, yy = meshgrid(arange(image.shape[0]), arange(image.shape[1]))

    if show:
        fig, axes = plt.subplots(len(diams), 2, figsize=(8,4*len(diams)), dpi=300)
        plt.suptitle(fname.split('/')[-1])

    for i, diam in enumerate(diams):
        masks = models.Cellpose(model_type='nuclei', gpu=True).eval(im, channels=[1,0], diameter=diam, cellprob_threshold=threshold)[0]

        if show:
            axes[i,0].imshow(im, origin='lower')
            axes[i,1].imshow(image, origin='lower')

        if i == 0:
            combined_masks = masks
        else:
            combined_masks[masks != 0] = masks[masks != 0] + combined_masks.max()

        im[:,:,0][binary_dilateNtimes(combined_masks, N=N_dilations) != 0] = 0
        im[:,:,0] = (255 * equalize_adapthist(im[:,:,0])).astype(uint8)

        if show:
            axes[i,0].contour(xx, yy, masks, colors='gold', levels=arange(masks.max()+1), linewidths=0.3)
            axes[i,1].contour(xx, yy, masks, colors='gold', levels=arange(masks.max()+1), linewidths=0.1)

            axes[i,0].tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
            axes[i,1].tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)

            axes[i,0].set_ylabel(f"{masks.max()} new masks @ D={diam} [pix]")
            axes[i,0].set_title("Input image"); axes[i,1].set_title("Original image")

    if show:
        plt.tight_layout()
        if save_to is not None:
            plt.savefig(save_to, bbox_inches='tight', dpi=300)
        plt.show()

    return combined_masks

In [ ]:
fname1 = '../control_images/001_z26.png'
fname2 = '../penetramax_images/011_z57.png'

_ = sequential_segmentation(fname1, diams=[80, 80, 80], threshold=0, show=True, save_to='../figures/control_segmentation.png')
_ = sequential_segmentation(fname2, diams=[80, 80, 80], threshold=0, show=True, save_to='../figures/penetramax_segmention.png')

In [ ]:
image1 = io.imread(fname1)
image2 = io.imread(fname2)

masks1 = sequential_segmentation(fname1, diams=[80, 80], threshold=0)
masks2 = sequential_segmentation(fname2, diams=[80, 80], threshold=0)

In [ ]:
plot_image_with_masks(image1, masks1), plot_image_with_masks(image2, masks2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

import os
from liams_funcs import *

from cellpose import models, io

In [ ]:
control_dir = '../control_images/'
penetramax_dir = '../penetramax_images/'

control_fnames = [control_dir + fname for fname in os.listdir(control_dir)]
penetramax_fnames = [penetramax_dir + fname for fname in os.listdir(penetramax_dir)]

In [ ]:
import os

# Directory paths
dir_hist = '../corrected_images/hist_correction/'
dir_adapthist = '../corrected_images/adapthist_correction/'

# Function to check if directory exists
def check_directory_exists(directory):
    try:
        if os.path.exists(directory) and os.path.isdir(directory):
            print(f"Directory exists: {directory}")
        else:
            raise NotADirectoryError(f"{directory} is not a valid directory.")
    except Exception as e:
        print(f"Error checking directory {directory}: {e}")

# Check if directories exist
check_directory_exists(dir_hist)
check_directory_exists(dir_adapthist)
check_directory_exists(penetramax_dir)
check_directory_exists(control_dir)

In [ ]:
diameters = np.arange(10, 200+1, 10)
model = models.Cellpose(model_type='nuclei', gpu=True)

N_segments = []
Areas = []
for diameter in tqdm(diameters):
    n_segments = []
    areas = []
    for fname in control_fnames[:1] + penetramax_fnames[:1]:
        image = io.imread(dir_hist + fname.split('/')[-1])

        masks = model.eval(image, channels=[1,0], diameter=diameter)[0]

        n_segments.append(masks.max())
        areas.append(np.sum(masks != 0))

    N_segments.append(n_segments)
    Areas.append(areas)

N_segments = np.asarray(N_segments)
Areas = np.asarray(Areas)

In [ ]:
fig, (l, r) = plt.subplots(1, 2, figsize=(12,4))

l.set_prop_cycle(plt.cycler('color', plt.cm.Reds(np.linspace(0.5, 1, len(control_fnames)))))
r.set_prop_cycle(plt.cycler('color', plt.cm.Blues(np.linspace(0.5, 1, len(control_fnames)))))


l.plot(diameters, N_segments, '-o')
l.plot(diameters, N_segments.mean(axis=1), c='aqua', linewidth=3)
l.set_yscale('log'); l.grid(axis='y')
l.set_xlim(diameters[0], diameters[-1])#; l.set_ylim(0)
l.set_yticks([10, 50, 100, 200, 500], [10, 50, 100, 200, 500])
l.set_xlabel('Diameter (pix)'); l.set_ylabel('# of segments')

r.plot(diameters, Areas, '-o')
r.plot(diameters, Areas.mean(axis=1), c='gold', linewidth=3)
r.set_yscale('log')
r.set_xlim(diameters[0], diameters[-1])#; r.set_ylim(0)
r.set_xlabel('Diameter (pix)'); r.set_ylabel(r'Area covered (pix$^2$)')

fig.tight_layout()
fig.savefig('diameter_analysis.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
def binary_dilateNtimes(mask, N:int=1):
    from skimage.morphology import dilation

    if N == 0:
        return mask
    return binary_dilateNtimes(dilation(mask), N=int(N-1))

def sequential_segmentation(fname, diams=[80, 80], N_dilations=5, threshold=0, show=False, save_to=None):
    from numpy import copy, uint8, arange, meshgrid
    from skimage.exposure import equalize_adapthist
    from cellpose import io

    image = io.imread(fname)
    im = copy(image)
    im[:,:,0] = (255 * equalize_adapthist(im[:,:,0])).astype(uint8)
    xx, yy = meshgrid(arange(image.shape[0]), arange(image.shape[1]))

    if show:
        fig, axes = plt.subplots(len(diams), 2, figsize=(8,4*len(diams)), dpi=300)
        plt.suptitle(fname.split('/')[-1])

    for i, diam in enumerate(diams):
        masks = models.Cellpose(model_type='nuclei', gpu=True).eval(im, channels=[1,0], diameter=diam, cellprob_threshold=threshold)[0]

        if show:
            axes[i,0].imshow(im, origin='lower')
            axes[i,1].imshow(image, origin='lower')

        if i == 0:
            combined_masks = masks
        else:
            combined_masks[masks != 0] = masks[masks != 0] + combined_masks.max()

        im[:,:,0][binary_dilateNtimes(combined_masks, N=N_dilations) != 0] = 0
        im[:,:,0] = (255 * equalize_adapthist(im[:,:,0])).astype(uint8)

        if show:
            axes[i,0].contour(xx, yy, masks, colors='gold', levels=arange(masks.max()+1), linewidths=0.3)
            axes[i,1].contour(xx, yy, masks, colors='gold', levels=arange(masks.max()+1), linewidths=0.1)

            axes[i,0].tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
            axes[i,1].tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)

            axes[i,0].set_ylabel(f"{masks.max()} new masks @ D={diam} [pix]")
            axes[i,0].set_title("Input image"); axes[i,1].set_title("Original image")

    if show:
        plt.tight_layout()
        if save_to is not None:
            plt.savefig(save_to, bbox_inches='tight', dpi=300)
        plt.show()

    return combined_masks

In [ ]:
def measure_coverage(nucleus, cell):
    """
    Measure the level of coverage between a nucleus and a cell.
    """

    return (nucleus * cell).sum() / nucleus.sum()

def cell_nucleus_segmentation(image, cell_diam=100, nucleus_diam=80, threshold=0, min_coverage=0.95):
    from cellpose.models import Cellpose
    from numpy import nan, ones, arange, stack

    # Predict the cell and nucleus masks:
    cells = Cellpose(model_type='cyto3', gpu=True).eval(image, channels=[3,1], diameter=cell_diam, cellprob_threshold=threshold)[0]
    nuclei = Cellpose(model_type='nuclei', gpu=True).eval(image, channels=[1,0], diameter=nucleus_diam, cellprob_threshold=threshold)[0]

    # Decompose from a 2D matrix to a list of 2D matrices, each with their own cell/nucleus:
    cell_masks = decompose_masks(cells, reduce=False)
    nucleus_masks = decompose_masks(nuclei, reduce=False)

    # Create a matrix, which stores the coverages between each cell and each nucleus:
    coverage_matrix = nan * ones(shape=(len(cell_masks), len(nucleus_masks)))
    for i, cell in enumerate(cell_masks):
        for j, nucleus in enumerate(nucleus_masks):
            coverage_matrix[i,j] = measure_coverage(nucleus, cell)

    # Interpret the matrix:
    # - If a cell completely covers a single nucleus, and doesn't collide with other nuclei, it passes the test
    unique_cells = []; unique_nuclei = []
    for i, cell in enumerate(cell_masks):
        column = coverage_matrix[i,:]

        if (column >= min_coverage).any():
            j = arange(len(column))[column == column.max()][0]
            
            unique_cells.append(i)
            unique_nuclei.append(j)

    # Combine into Cellpose-like outputs:
    unique_cell_masks = []
    unique_nucleus_masks = []
    count = 1
    for cell_idx, nucleus_idx in zip(unique_cells, unique_nuclei):
        unique_cell_masks.append(count * cell_masks[cell_idx])
        unique_nucleus_masks.append(count * nucleus_masks[nucleus_idx])

        count = int(count + 1)

    unique_cell_masks = stack(unique_cell_masks, axis=2).sum(axis=2)
    unique_nucleus_masks = stack(unique_nucleus_masks, axis=2).sum(axis=2)

    return unique_cell_masks, unique_nucleus_masks

def combined_segmentation(fname, cell_diams=[100, 100], nucleus_diams=[80, 80], threshold=0, show=False, correction=perform_adapthist_equalisation, min_coverage=0.99, save_to=None):
    from cellpose.io import imread
    from numpy import copy, stack, arange, meshgrid

    image = imread(fname)
    im = correction(copy(image))
    xx, yy = meshgrid(arange(image.shape[0]), arange(image.shape[1]))

    if show:
        fig, axes = plt.subplots(min([len(cell_diams), len(nucleus_diams)]), 2, figsize=(8,4*min([len(cell_diams), len(nucleus_diams)])), dpi=300)
        plt.suptitle(fname.split('/')[-1])
    
    for i, (cell_diam, nucleus_diam) in enumerate(zip(cell_diams, nucleus_diams)):
        cells, nuclei = cell_nucleus_segmentation(im, cell_diam=cell_diam, nucleus_diam=nucleus_diam, threshold=threshold, min_coverage=min_coverage)
        print(f"{cells.max()} new cell/nucleus pairs")

        if show:
            axes[i,0].imshow(im, origin='lower')
            axes[i,1].imshow(image, origin='lower')

        if i == 0:
            combined_cells = cells
            combined_nuclei = nuclei
        else:
            combined_cells[cells != 0] = cells[cells != 0] + combined_cells.max()
            combined_nuclei[nuclei != 0] = nuclei[nuclei != 0] + combined_nuclei.max()

        mask = (combined_cells != 0)
        channels = []
        for j in range(im.shape[2]):
            colour = im[:,:,j]
            colour[mask] = 0
            channels.append(colour)

        im = correction(stack(channels, axis=2))

        if show:
            axes[i,0].contour(xx, yy, cells, colors='fuchsia', levels=arange(combined_cells.max()+1), linewidths=0.3)
            axes[i,0].contour(xx, yy, nuclei, colors='gold', levels=arange(combined_nuclei.max()+1), linewidths=0.3)
            axes[i,1].contour(xx, yy, combined_cells, colors='fuchsia', levels=arange(combined_cells.max()+1), linewidths=0.3)
            axes[i,1].contour(xx, yy, combined_nuclei, colors='gold', levels=arange(combined_nuclei.max()+1), linewidths=0.3)

            axes[i,0].tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
            axes[i,1].tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)

            axes[i,0].set_ylabel(f"{nuclei.max()} new masks")
            axes[i,0].set_title("Input image"); axes[i,1].set_title("Original image")

    if show:
        plt.tight_layout()
        if save_to is not None:
            plt.savefig(save_to, bbox_inches='tight', dpi=300)
        plt.show()

    return combined_cells, combined_nuclei

In [ ]:
fname1 = '../control_images/001_z26.png'
fname2 = '../penetramax_images/011_z57.png'

In [ ]:
_ = sequential_segmentation(fname1, diams=[80, 80, 80], threshold=0, show=True, save_to='../figures/control_segmentation.png')
_ = sequential_segmentation(fname2, diams=[80, 80, 80], threshold=0, show=True, save_to='../figures/penetramax_segmention.png')

In [ ]:
_ = combined_segmentation(fname1, cell_diams=[100, 100, 100], nucleus_diams=[80, 80, 80], show=True, min_coverage=0.95, save_to='../figures/combined_segmentation_95.png')
_ = combined_segmentation(fname1, cell_diams=[100, 100, 100], nucleus_diams=[80, 80, 80], show=True, min_coverage=0.97, save_to='../figures/combined_segmentation_97.png')
_ = combined_segmentation(fname1, cell_diams=[100, 100, 100], nucleus_diams=[80, 80, 80], show=True, min_coverage=0.99, save_to='../figures/combined_segmentation_99.png')

In [ ]:
_ = combined_segmentation(fname1, show=True, save_to='../figures/control_combined_segmentation.png')
_ = combined_segmentation(fname2, show=True, save_to='../figures/penetramax_combined_segmentation1.png')

In [ ]:
import os
import numpy as np

def process_files(fnames, dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
        
    for fname in fnames:
        try:
            print(fname.split('/')[-1])
            new_fname = os.path.join(dir, fname.split('/')[-1])
            
            combined_cells, combined_nuclei = combined_segmentation(fname)
            data = np.stack([combined_cells, combined_nuclei], axis=2)
            np.save(new_fname, data)
        except FileNotFoundError:
            print(f"File not found: {fname}")
        except Exception as e:
            print(f"An error occurred while processing {fname}: {e}")

dir_control = '../segments/control/'
process_files(control_fnames, dir_control)

dir_penetramax = '../segments/penetramax/'
process_files(penetramax_fnames, dir_penetramax)


In [ ]:
def normalise(arr):
    return (arr - arr.min()) / (arr.max() - arr.min())

In [ ]:
data = np.load('../segments/control/001_z26.png.npy')
image = io.imread('../control_images/001_z26.png')

reds, blues = image[:,:,0], image[:,:,2]
combined_cells, combined_nuclei = data[:,:,0], data[:,:,1]

In [ ]:
qs = np.linspace(0, 1, 20, endpoint=True)
vals = np.arange(1, combined_nuclei.max()+1)

fig, (l, r) = plt.subplots(1, 2, figsize=(12,4), dpi=300)

l.set_prop_cycle(plt.cycler('color', plt.cm.Reds(np.linspace(0.5, 1, len(vals)))))
r.set_prop_cycle(plt.cycler('color', plt.cm.Blues(np.linspace(0.5, 1, len(vals)))))

for val in vals:
    mask = (combined_nuclei == val)
    red = reds[mask]
    blue = blues[mask]

    l.plot(np.quantile(red, qs), qs, alpha=0.2)
    r.plot(np.quantile(blue, qs), qs, alpha=0.2)

l.set_xscale('log', base=2); r.set_xscale('log', base=2)
l.set_ylim(0, 1); r.set_ylim(0, 1)
l.set_xlim(2, 256); r.set_xlim(2, 256)

plt.show()

In [ ]:
class Masks:
    def __init__(self, fname):
        from skimage.measure import regionprops

        if fname in os.listdir('../control_images/'):
            self.image_dir = '../control_images/'
            self.segment_dir = '../segments/control/'
        else:
            self.image_dir = '../penetramax_images/'
            self.segment_dir = '../segments/penetramax/'

        self.image = io.imread(self.image_dir + fname)
        data = np.load(self.segment_dir + fname + '.npy')

        self.cells = data[:,:,0]; self.nuclei = data[:,:,1]
        self.reds = self.image[:,:,0]; self.blues = self.image[:,:,1]

        self.vals = np.arange(1, self.cells.max()+1)

    def getRedness(self, idx=1, N=10, q_lim=(0.05, 0.95), rescaling=np.log2, normalisation=normalise):
        val = self.vals[idx]
        nucleus = (self.nuclei == val)
        red_array = self.reds[nucleus]

        qs = np.linspace(q_lim[0], q_lim[1], N, endpoint=True)
        red_quantiles = np.quantile(red_array, qs)

        return normalisation(rescaling(red_quantiles))

    def getBlueness(self, idx=1, N=10, q_lim=(0.05, 0.95), rescaling=np.log2, normalisation=normalise):
        val = self.vals[idx]
        nucleus = (self.nuclei == val)
        blue_array = self.blues[nucleus]

        qs = np.linspace(q_lim[0], q_lim[1], N, endpoint=True)
        blue_quantiles = np.quantile(blue_array, qs)

        return normalisation(rescaling(blue_quantiles))
    
    def getRoundness(self, idx=1, rescaling=lambda x: x):
        from liams_funcs import evaluate_roundness
        
        val = self.vals[idx]
        nucleus = (self.nuclei == val)
        cell = (self.cells == val)

        return rescaling(evaluate_roundness(nucleus)), rescaling(evaluate_roundness(cell))
    
    def getAllProperties(self, idx=1):
        x1 = list(self.getRoundness(idx=idx))
        x2 = list(self.getRedness(idx=idx))
        x3 = list(self.getBlueness(idx=idx))
        return np.array(x1 + x2 + x3)
    
    def getX(self):
        X = []
        for idx in range(len(self.vals)):
            X.append(self.getAllProperties(idx=idx))
        return np.stack(X, axis=0)

In [ ]:
df = {}

for fname in tqdm(control_fnames + penetramax_fnames):
    mask = Masks(fname.split('/')[-1])
    X = mask.getX()

    for count in range(X.shape[0]):
        df[f"{count}_{fname.split('/')[-1]}"] = X[count,:]

df = pd.DataFrame().from_dict(df)

In [ ]:
X = df.to_numpy().T
X.shape

In [ ]:
from sklearn.decomposition import PCA

X_PCA = PCA(n_components=2).fit_transform(X)

In [ ]:
from scipy.stats import gaussian_kde

kde = gaussian_kde(X_PCA.T)

N = 100
xx, yy = np.meshgrid(np.linspace(-1, 1.5, N, endpoint=True), np.linspace(-1, 1.5, N, endpoint=True))
coords = np.stack([xx.reshape(N**2), yy.reshape(N**2)])
zz = np.log10(kde(coords).reshape(100, 100))

fig, ax = plt.subplots(figsize=(12,4), dpi=300)
ax.set_aspect('equal')

ax.contour(xx, yy, zz, colors='k', levels=(zz.max() - np.arange(0, 5))[::-1])
ax.contourf(xx, yy, zz, cmap='Reds', levels=(zz.max() - np.arange(0, 10))[::-1])
ax.scatter(X_PCA[:,0], X_PCA[:,1], s=1, alpha=0.5, c='gold')

plt.show()

In [ ]:
plt.plot(X[:,0], X[:,1], '.')